In [1]:
#import relevant packages
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt


In [2]:
#use jmc's code for pulling data in

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)
 
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Grocery_and_Gourmet_Food_5.json.gz')
meta=getDF('meta_Grocery_and_Gourmet_Food.json.gz')
stars=pd.read_csv('ratings_Grocery_and_Gourmet_Food.csv')
stars.head()

,A1ZQZ8RJS1XVTX,0657745316,5.0,1381449600
0,A31W38VGZAUUM4,0700026444,5.0,1354752000
1,A3I0AV0UJX5OH0,1403796890,1.0,1385942400
2,A3QAAOLIXKV383,1403796890,3.0,1307836800
3,AB1A5EGHHVA9M,141278509X,5.0,1332547200
4,A3DTB6RVENLQ9Q,1453060375,1.0,1362268800


In [3]:
stars=pd.read_csv('ratings_Grocery_and_Gourmet_Food.csv',names=['reviewerID','asin','overall','unixReviewTime'])
print(len(stars))
meanstars=stars.groupby('asin').agg('mean')
meanstars['asin']=meanstars.index
meanstars['meanstars']=meanstars['overall']
#meanstars=meanstars[['asin','meanstars']].reset_index()
meanstars=meanstars[['asin','meanstars']]
meanstars.head()

1297156


,asin,meanstars
asin,,
0657745316,0657745316,5.0
0700026444,0700026444,5.0
1403796890,1403796890,2.0
141278509X,141278509X,5.0
1453060375,1453060375,1.0


In [ ]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [ ]:
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)


### merge reviews and meta dfs
WHAT'S POSITIVE OR NEGATIVE? 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151254 entries, 0 to 151253
Data columns (total 9 columns):
summary           151254 non-null object
overall           151254 non-null float64
unixReviewTime    151254 non-null int64
reviewTime        151254 non-null object
helpful           151254 non-null object
reviewText        151254 non-null object
reviewerName      149761 non-null object
asin              151254 non-null object
reviewerID        151254 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 11.5+ MB


In [5]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171760 entries, 0 to 171759
Data columns (total 9 columns):
imUrl          171616 non-null object
categories     171760 non-null object
description    151817 non-null object
title          171602 non-null object
related        127270 non-null object
asin           171760 non-null object
salesRank      156972 non-null object
price          113152 non-null float64
brand          63905 non-null object
dtypes: float64(1), object(8)
memory usage: 13.1+ MB


In [6]:
metadf=pd.merge(meta,df,on='asin',how='outer')
print(len(meta),len(df),len(metadf))
metadf.head()
metadfmean=pd.merge(metadf,meanstars[['asin']],on='asin',how='outer')
print(len(meta),len(df),len(metadf),len(meanstars),len(metadfmean))
metadfmean.head()

171760 151254 314301
171760 151254 314301 166049 314301


,related,categories,description,imUrl,asin,title,salesRank,price,brand,overall,reviewerName,reviewText,summary,helpful,reviewerID,unixReviewTime,reviewTime
0,{'also_viewed': ['B001GE8N4Y']},[[Grocery & Gourmet Food]],This is real vanilla extract made with only 3 ...,http://ecx.images-amazon.com/images/I/41gFi5h0...,0657745316,100 Percent All Natural Vanilla Extract,{'Grocery & Gourmet Food': 374004},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,[[Grocery & Gourmet Food]],"Silverpot Tea, Pure Darjeeling, is an exquisit...",http://ecx.images-amazon.com/images/I/51hs8sox...,0700026444,Pure Darjeeling Tea: Loose Leaf,{'Grocery & Gourmet Food': 620307},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,[[Grocery & Gourmet Food]],Must have for any WWE Fan\n \n \n \nFeaturing ...,http://ecx.images-amazon.com/images/I/518SEST5...,1403796890,WWE Kids Todler Velvet Slippers featuring John...,NaN,3.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'also_viewed': ['B0051IETTY']},[[Grocery & Gourmet Food]],Infused with Vitamins and Electrolytes Good So...,http://ecx.images-amazon.com/images/I/51CFQIis...,141278509X,Archer Farms Strawberry Dragonfruit Drink Mix ...,{'Grocery & Gourmet Food': 620322},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'also_viewed': ['B006MSEOJ2', 'B005VOOQLO', '...",[[Grocery & Gourmet Food]],MiO Energy is your portable energy source givi...,http://ecx.images-amazon.com/images/I/51EUsMcn...,1453060375,Mio Energy Liquid Water Enhancer Black Cherry ...,{'Grocery & Gourmet Food': 268754},11.99,Mio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
metadfmean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314301 entries, 0 to 314300
Data columns (total 17 columns):
related           264646 non-null object
categories        314301 non-null object
description       290900 non-null object
imUrl             313981 non-null object
asin              314301 non-null object
title             313967 non-null object
salesRank         292953 non-null object
price             232520 non-null float64
brand             162188 non-null object
overall           151254 non-null float64
reviewerName      149761 non-null object
reviewText        151254 non-null object
summary           151254 non-null object
helpful           151254 non-null object
reviewerID        151254 non-null object
unixReviewTime    151254 non-null float64
reviewTime        151254 non-null object
dtypes: float64(3), object(14)
memory usage: 43.2+ MB


In [8]:
metadfmean.to_csv('amawed.csv')

In [44]:
for x in [meta,df, metadf,metadfmean]:
    print(x.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171760 entries, 0 to 171759
Data columns (total 9 columns):
imUrl          171616 non-null object
categories     171760 non-null object
description    151817 non-null object
title          171602 non-null object
related        127270 non-null object
asin           171760 non-null object
salesRank      156972 non-null object
price          113152 non-null float64
brand          63905 non-null object
dtypes: float64(1), object(8)
memory usage: 13.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 151254 entries, 0 to 151253
Data columns (total 9 columns):
summary           151254 non-null object
overall           151254 non-null float64
unixReviewTime    151254 non-null int64
reviewTime        151254 non-null object
helpful           151254 non-null object
reviewText        151254 non-null object
reviewerName      149761 non-null object
asin              151254 non-null object
reviewerID        151254 non-null object
dtypes: flo

In [24]:
#GET A LIST OF BRANDS TO FACILITATE MERGING WITH USDA
st=[]
for b in metadfmean['brand'][metadfmean.brand.notnull()]:
    st.append(b.replace('&#39;',"'").replace('&reg;','').replace('&amp;','&'))
#metadfmean['cleanbrands']=st
#metadfmean.info()

In [44]:
#GET A LIST OF BRANDS TO FACILITATE MERGING WITH USDA
st=[]
for b in metadfmean['brand']:
    if type(b) is float:
        st.append(b)
    elif type(b) is not float:
        st.append(b.replace('&#39;',"'").replace('&reg;','').replace('&amp;','&'))
metadfmean['cleanbrands']=st
metadfmean.head()

,related,categories,description,imUrl,asin,title,salesRank,price,brand,overall,reviewerName,reviewText,summary,helpful,reviewerID,unixReviewTime,reviewTime,cleanbrands
0,{'also_viewed': ['B001GE8N4Y']},[[Grocery & Gourmet Food]],This is real vanilla extract made with only 3 ...,http://ecx.images-amazon.com/images/I/41gFi5h0...,0657745316,100 Percent All Natural Vanilla Extract,{'Grocery & Gourmet Food': 374004},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,[[Grocery & Gourmet Food]],"Silverpot Tea, Pure Darjeeling, is an exquisit...",http://ecx.images-amazon.com/images/I/51hs8sox...,0700026444,Pure Darjeeling Tea: Loose Leaf,{'Grocery & Gourmet Food': 620307},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,[[Grocery & Gourmet Food]],Must have for any WWE Fan\n \n \n \nFeaturing ...,http://ecx.images-amazon.com/images/I/518SEST5...,1403796890,WWE Kids Todler Velvet Slippers featuring John...,NaN,3.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'also_viewed': ['B0051IETTY']},[[Grocery & Gourmet Food]],Infused with Vitamins and Electrolytes Good So...,http://ecx.images-amazon.com/images/I/51CFQIis...,141278509X,Archer Farms Strawberry Dragonfruit Drink Mix ...,{'Grocery & Gourmet Food': 620322},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'also_viewed': ['B006MSEOJ2', 'B005VOOQLO', '...",[[Grocery & Gourmet Food]],MiO Energy is your portable energy source givi...,http://ecx.images-amazon.com/images/I/51EUsMcn...,1453060375,Mio Energy Liquid Water Enhancer Black Cherry ...,{'Grocery & Gourmet Food': 268754},11.99,Mio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mio


In [45]:
metadfmean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314301 entries, 0 to 314300
Data columns (total 18 columns):
related           264646 non-null object
categories        314301 non-null object
description       290900 non-null object
imUrl             313981 non-null object
asin              314301 non-null object
title             313967 non-null object
salesRank         292953 non-null object
price             232520 non-null float64
brand             162188 non-null object
overall           151254 non-null float64
reviewerName      149761 non-null object
reviewText        151254 non-null object
summary           151254 non-null object
helpful           151254 non-null object
reviewerID        151254 non-null object
unixReviewTime    151254 non-null float64
reviewTime        151254 non-null object
cleanbrands       162188 non-null object
dtypes: float64(3), object(15)
memory usage: 45.6+ MB


In [52]:
metadfmean.description[0]

'This is real vanilla extract made with only 3 premium ingredients. GMO free, no fillers you find in store bought "vanilla extract." \n\nThe taste will knock your socks off. Everyone will notice a difference in your baking and cooking and they\'ll want to know your secret. I also use this for a special homemade coffee creamer that\'s out of this world and I use it for tea and black coffee as well as espresso drinks. You can add this to make a vanilla latte and skip the sugary syrup for a healthier latte with more flavor! \n\nWhen this item arrives, there will also be instructions to refill the product as its used so that you won\'t have to age it or repurchase it. I\'ve been using the same vanilla for 2 years now and have friends who\'ve had theirs for 5 years. It\'s just as tasteful, just as sweet, strong, and yummy. \n\nI use only top shelf liquor and the product is aged a minimum of 4 months. \n\nThese also make great gifts. I currently have plenty on the shelf but they won\'t be re

In [54]:
metadfmean.iloc[0]

related                                                                               {'also_viewed': ['B001GE8N4Y']}
categories                                                                                 [[Grocery & Gourmet Food]]
description       This is real vanilla extract made with only 3 premium ingredients. GMO free, no fillers you find...
imUrl                                                   http://ecx.images-amazon.com/images/I/41gFi5h0jYL._SY300_.jpg
asin                                                                                                       0657745316
title                                                                         100 Percent All Natural Vanilla Extract
salesRank                                                                          {'Grocery & Gourmet Food': 374004}
price                                                                                                             NaN
brand                                                   

In [64]:
seram=[]
for i in metadfmean.categories:#[metadfmean.categories.notnull()]:
    seram.append(i[0])
 
seram[:10]

[['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food'],
 ['Grocery & Gourmet Food', 'Beverages', 'Coffee', 'Coffee Substitutes'],
 ['Grocery & Gourmet Food']]

In [78]:
seramdf=pd.DataFrame(seram,columns=['Maincat','cat','cat3_122','cat4_190','cat5','cat6'])
seramdf.head()

,Maincat,cat,cat3_122,cat4_190,cat5,cat6
0,Grocery & Gourmet Food,None,None,None,None,None
1,Grocery & Gourmet Food,None,None,None,None,None
2,Grocery & Gourmet Food,None,None,None,None,None
3,Grocery & Gourmet Food,None,None,None,None,None
4,Grocery & Gourmet Food,None,None,None,None,None


In [81]:
amause=metadfmean.join(seramdf)

In [88]:
amause.groupby('brand')['title'].count()

brand
                                                   875
&Uuml;lker                                          10
&aacute;vitae                                      328
&amp; Salty Popcorn Chips Baked Snacks (Pack 4)      1
---                                                  1
100 Calorie Packs                                   22
100 Grand                                            3
10K                                                  4
123 Gluten Free                                     10
17th Street BBQ                                      1
18 Rabbits                                          10
180 Snacks                                           7
1883                                                 4
1907                                                 1
1st Endurance                                        3
200000 Mg Glutathione                                1
21st Century                                         2
21st century snacks                                  2
22 D

In [93]:
amb=amause.cleanbrands.value_counts().index 
amb[:5]

Index(['Bob's Red Mill', 'Green Mountain Coffee', 'Lipton', 'Barilla',
       'Nutiva'],
      dtype='object')

In [95]:
pd.DataFrame(amb).to_csv('amb.csv')

In [ ]:

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [32]:
mhpdf.categories.iloc[0].

[['Health & Personal Care',
  'Medical Supplies & Equipment',
  'Daily Living Aids',
  'Visual Impairment Aids',
  'Magnifiers']]

In [86]:
seram=[]
for i in mhpdf.categories[mhpdf.categories.notnull()]:
        seram.append(i[0])
 
len(seram)


346355

In [88]:
seramdf=pd.DataFrame(seram,columns=['H&PC','Cat1','Cat2','Cat3','Cat4','Cat5','Cat6'],index=mhpdf.index)
seramdf.head()

,H&PC,Cat1,Cat2,Cat3,Cat4,Cat5,Cat6
0,Health & Personal Care,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
1,Health & Personal Care,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
2,Health & Personal Care,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
3,Health & Personal Care,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
4,Health & Personal Care,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None


In [91]:
seramdf['H&PC'].value_counts()

Health & Personal Care       345755
Automotive                      469
Cell Phones & Accessories        61
Sports & Outdoors                59
Home & Kitchen                    6
CDs & Vinyl                       5
Name: H&PC, dtype: int64

In [101]:
seramdf[seramdf['H&PC']=='Automotive'].Cat2.value_counts()

Tools & Equipment        410
Paints & Primers          27
Exterior Care             24
Electrical Appliances      8
Name: Cat2, dtype: int64

In [102]:
seramdf['Cat1'].value_counts()

Vitamins & Dietary Supplements    83181
Personal Care                     73828
Health Care                       59636
Household Supplies                59060
Medical Supplies & Equipment      35702
Sports Nutrition                  20787
Sexual Wellness                    6276
Stationery & Party Supplies         771
Baby & Child Care                   634
Car Care                            434
Accessories                          90
Paint, Body & Trim                   27
Clothing                             22
Interior Accessories                  8
Outdoor Gear                          8
Kitchen & Dining                      6
New Age                               5
Name: Cat1, dtype: int64

In [111]:
mhpdfa=pd.merge(seramdf,mhpdf,left_index=True,right_index=True)

In [114]:
mhpdfa.columns

Index(['H&PC', 'Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6', 'reviewerName',
       'reviewTime', 'summary', 'reviewText', 'unixReviewTime', 'asin',
       'reviewerID', 'overall', 'helpful', 'description', 'categories',
       'related', 'title', 'salesRank', 'imUrl', 'price', 'brand'],
      dtype='object')

In [120]:
mhpdfa[['Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6']][:5]

,Cat1,Cat2,Cat3,Cat4,Cat5,Cat6
0,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
1,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
2,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
3,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None
4,Medical Supplies & Equipment,Daily Living Aids,Visual Impairment Aids,Magnifiers,None,None


In [121]:
mhpdfa.Cat1.value_counts()

Vitamins & Dietary Supplements    83181
Personal Care                     73828
Health Care                       59636
Household Supplies                59060
Medical Supplies & Equipment      35702
Sports Nutrition                  20787
Sexual Wellness                    6276
Stationery & Party Supplies         771
Baby & Child Care                   634
Car Care                            434
Accessories                          90
Paint, Body & Trim                   27
Clothing                             22
Interior Accessories                  8
Outdoor Gear                          8
Kitchen & Dining                      6
New Age                               5
Name: Cat1, dtype: int64

In [175]:
colsdrop=['Vitamins & Dietary Supplements','Medical Supplies & Equipment','Sports Nutrition',
         'Stationery & Party Supplies','Interior Accessories','Clothing','Outdoor Gear',
         'New Age','Accessories','Baby & Child Care','Health Care','Kitchen & Dining']
mhpdfacat1=mhpdfa[~mhpdfa['Cat1'].isin(colsdrop)]

In [ ]:
colsdrop2=['Cleaning Tools','Household Batteries','Paper & Plastic','Adult Toys & Games',
           'Bondage Gear & Accessories','Fetish Wear','Tools & Equipment','Novelties','Sexual Enhancers']
mhpdfacat1_=mhpdfacat1[~mhpdfacat1['Cat2'].isin(colsdrop2)]
colsdrop3=['Deodorants','Alternative Protection','Panty Liners'          
'Reading Glasses','Eyeglasses Care','Contact Lens Care','Manual Shaving','Electric Shavers',  
'Trimmers & Clippers','Shaving Sets & Kits','Shaving Accessories','Cloths & Wipes']
mhpdfacat2_=mhpdfacat1_[~mhpdfacat1_['Cat3'].isin(colsdrop3)]


In [213]:
mhpdfacat2_.Cat.value_counts()

Oral Hygiene                    15338
Shaving & Hair Removal          12464
Laundry                         10725
Household Cleaning               8419
Dishwashing                      4668
Deodorants & Antiperspirants     4432
Feminine Care                    3077
Lip Care                         2543
Eye Care                         2482
Safer Sex                        2198
Air Fresheners                   2161
Foot Care                        1946
Ear Care                         1698
Body Art                          100
Paints & Primers                   27
Exterior Care                      24
Sensual Delights                   16
Name: Cat2, dtype: int64

In [220]:
mhpdfacat2_.to_csv('amazon.csv')

In [211]:
mhpdfacat1[mhpdfacat1.Cat2=='Household Cleaning']['Cat3'].value_counts()

All-Purpose Cleaners             1665
Bathroom Cleaners                1655
Cloths & Wipes                   1461
Carpet Cleaners & Deodorizers    1350
Floor Cleaners                    917
Kitchen Cleaners                  732
Furniture & Wood Polishes         500
Drain Openers                     447
Metal Polishes                    361
Glass Cleaners                    297
Upholstery Cleaners               274
Name: Cat3, dtype: int64

In [1]:
import pandas as pd
import numpy as np

In [2]:
am=pd.read_csv('amazon.csv')

In [46]:
am.Cat2.value_counts()

Oral Hygiene                    15338
Shaving & Hair Removal          12464
Laundry                         10725
Household Cleaning               8419
Dishwashing                      4668
Deodorants & Antiperspirants     4432
Feminine Care                    3077
Lip Care                         2543
Eye Care                         2482
Safer Sex                        2198
Air Fresheners                   2161
Foot Care                        1946
Ear Care                         1698
Body Art                          100
Paints & Primers                   27
Exterior Care                      24
Sensual Delights                   16
Name: Cat2, dtype: int64

In [47]:
4241*15000

63615000

In [48]:
am.head()

,Unnamed: 0,H&PC,Cat1,Cat2,Cat3,Cat4,Cat5,Cat6,reviewerName,reviewTime,...,overall,helpful,description,categories,related,title,salesRank,imUrl,price,brand
0,80,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,adero1780,"01 11, 2013",...,5.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
1,81,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,Allan Hamilton,"07 9, 2014",...,5.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
2,82,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,Amazon Customer,"11 28, 2013",...,4.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
3,83,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,Amazon Customer,"08 4, 2011",...,1.0,"[1, 1]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
4,84,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,"Amazon Customer ""mikeklement.com""","11 27, 2012",...,5.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer


In [49]:
am.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81120 entries, 0 to 81119
Data columns (total 25 columns):
Unnamed: 0        81120 non-null int64
H&PC              81120 non-null object
Cat1              75240 non-null object
Cat2              72318 non-null object
Cat3              69758 non-null object
Cat4              19550 non-null object
Cat5              900 non-null object
Cat6              0 non-null float64
reviewerName      80714 non-null object
reviewTime        81120 non-null object
summary           81120 non-null object
reviewText        81103 non-null object
unixReviewTime    81120 non-null int64
asin              81120 non-null object
reviewerID        81120 non-null object
overall           81120 non-null float64
helpful           81120 non-null object
description       56503 non-null object
categories        81120 non-null object
related           79362 non-null object
title             80691 non-null object
salesRank         77385 non-null object
imUrl            

In [51]:
am.title.value_counts()

Tide Original Scent Liquid Laundry Detergent ,  50 Fl Oz,  2 Count                                                                               1089
Secret Clinical Strength Sport Advanced Solid Antiperspirant &amp; Deodorant Marathon Fresh Scent 1.6 Oz (Pack of 2)                              469
Secret Clinical Strength Advanced Solid Antiperspirant &amp; Deodorant Light And Fresh Scent 1.6 Oz (Pack of 2)                                   431
Tide Pods Laundry Detergent Alpine Breeze Scent 18 Count                                                                                          427
Crest 3D White Whitestrips Professional Effects - Teeth Whitening Kit 20 Treatments (Packaging May Vary)                                          321
Proraso Shaving Cream, Eucalyptus &amp; Menthol, 150 ml, New Formulation                                                                          298
Proraso Shaving Soap, Eucalyptus &amp; Menthol, 5.2 oz (150 ml), New Formulation                    

In [9]:
am.head()

,Unnamed: 0,H&PC,Cat1,Cat2,Cat3,Cat4,Cat5,Cat6,reviewerName,reviewTime,...,overall,helpful,description,categories,related,title,salesRank,imUrl,price,brand
0,80,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,adero1780,"01 11, 2013",...,5.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
1,81,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,Allan Hamilton,"07 9, 2014",...,5.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
2,82,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,Amazon Customer,"11 28, 2013",...,4.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
3,83,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,Amazon Customer,"08 4, 2011",...,1.0,"[1, 1]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer
4,84,Health & Personal Care,Household Supplies,NaN,NaN,NaN,NaN,NaN,"Amazon Customer ""mikeklement.com""","11 27, 2012",...,5.0,"[0, 0]",Energizer Recharge Batteries are the reusable ...,"[['Health & Personal Care', 'Household Supplie...","{'bought_together': ['B000BESLQK', 'B00339NINQ...",Energizer Power Plus NiMH AA Rechargeable Batt...,{'Home Improvement': 541},http://ecx.images-amazon.com/images/I/61g6ibWk...,10.42,Energizer


In [ ]:
'Oral Irrigators','Power Toothbrushes','Dental Floss & Flossers','Dental Picks','Oral Irrigators',
'Manual Toothbrushes','Gum Stimulators','Tampons','Sanitary Napkins','Panty Liners',
'Callus Shavers','Scrubs, Salts & Soaks','Callus Stones','Moisturizing Socks','Earplugs          1143
'Earwax Removal','Ear Drops','Ear Syringes'

# Natural Language Processing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
# Thanks
To Julian McAuly for permitting use of data scraped by him/his team

WWW / SIGIR papers  
R. He, J. McAuley. Modeling the visual evolution of fashion trends
with one-class collaborative filtering. WWW, 2016
J. McAuley, C. Targett, J. Shi, A. van den Hengel. Image-based
recommendations on styles and substitutes. SIGIR, 2015